In [1]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
import numpy as np

import matplotlib.pyplot as plt

In [2]:
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev4/ao0:3", min_val=-10, max_val=100)
task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
xy_val = 3
xy = np.linspace(-xy_val,xy_val,200)
t =  np.zeros(200)
t[0] = 5
t0 = t.copy()
for i in range(50):
    xy = np.append(xy, np.linspace(xy_val,-xy_val,200))
    xy = np.append(xy, np.linspace(-xy_val,xy_val,200))
    t = np.append(t, t0)
    t = np.append(t, t0)
    
#xy = np.concatenate([np.linspace(-1,1,200), np.linspace(1,-1,200)])
z = np.linspace(-1, 1, 200*101)
p = np.linspace(5, 0, 200*100 - 800)
p = np.append(p, np.linspace(0, 5, 1000))
              
print(np.shape(z))
print(np.shape(p))
#p = np.linspace(5, 0, 200*101)
array = np.stack([xy, z, p, t])

#plt.figure()
plt.plot(array.T)
plt.xlim(0,500)

In [ ]:
task.write(array, auto_start=True)

In [ ]:
task.stop()

In [ ]:
np.max(xy)

In [ ]:
import tifffile as tiff
from pathlib import Path

In [ ]:
path = Path(r"C:\Users\Admin\Documents\mpImage(61).tif")
stack = tiff.imread(path)

In [ ]:
np.shape(stack)

### Calibration

In [ ]:
piezo1 = 4
z_galvo1 = 0

xy_set = xy
z_set = np.ones(200*100) * z_galvo1
p_set = np.ones(200*100) * piezo1
array_set = np.stack([xy_set, z_set])
task.write(array_set, auto_start=True)
task.stop()

In [ ]:
piezo2 = 0
z_galvo2 = -0.8

xy_set = xy
z_set = np.ones(200*100) * z_galvo2
p_set = np.ones(200*100) * piezo2
array_set = np.stack([xy_set, z_set, p_set])
task.write(array_set, auto_start=True)
task.stop()

In [ ]:
piezo3 = 3
z_galvo3 = 0

xy_set = xy
z_set = np.ones(200*100) * z_galvo3
p_set = np.ones(200*100) * piezo3
array_set = np.stack([xy_set, z_set, p_set])
task.write(array_set, auto_start=True)
task.stop()

In [ ]:
piezo_calibration = np.asarray([piezo1, piezo2, piezo3])
z_calibration = np.asarray([z_galvo1, z_galvo2, z_galvo3])

In [ ]:
calibration_data = np.zeros((3,3))
calibration_data[:, 0] = piezo_calibration.T
calibration_data[:, 1] = z_calibration.T
piezo_val = np.pad(
    calibration_data[:, 0:1],
    ((0, 0), (1, 0)),
    constant_values=1.0,
    mode="constant",
)
galvo = calibration_data[:, 1]

In [ ]:
# solve least squares according to standard formula b = (XtX)^-1 * Xt * y
piezo_cor = np.linalg.pinv(piezo_val.T @ piezo_val)
calibration = piezo_cor @ piezo_val.T @ galvo

In [ ]:
calibration

## Simpler calibration

In [ ]:
xy = [np.linspace(-0.4,0.4,200)]
for i in range(99):
    xy = np.append(xy, np.linspace(-0.4,0.4,200))
z = np.linspace(z_galvo1, z_galvo2, 200*100)
p = np.linspace(piezo1, piezo2, 190*100)
p = np.append(p, np.linspace(piezo2, piezo1, 1000))
              
print(np.shape(xy))
print(np.shape(z))
array_set = np.stack([xy, z, p])

In [ ]:
task.write(array_set, auto_start=True)

In [ ]:
task.stop()